In [1]:
import pandas as pd
import numpy as np

In [2]:
pip install pandas numpy scikit-learn fuzzywuzzy python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.1 MB/s eta 0:00:00


In [3]:
from fuzzywuzzy import process
from sklearn.preprocessing import MinMaxScaler

In [4]:
df = pd.read_csv('emobile.csv')

In [5]:
df.head()

,pid,pname,pratings,nop
0,p1254,samsung mobile-g56,8.3,1214
1,p1682,samsung mobile-f56,6.2,821
2,p1852,vivo-1213,4.2,501
3,p1987,samsung mobile-h56,5.4,623
4,p1365,vivo-v1-pro,8.3,1211


In [6]:
dx = pd.read_csv('maxmobile.csv')

In [7]:
df.drop_duplicates(inplace=True)
dx.drop_duplicates(inplace=True)

In [8]:
df.columns=['Product_id','Product_Name','Rating', 'Review_Count']

In [9]:
dx.columns=['Product_id','Product_Name','Rating', 'Review_Count']

In [22]:
def match_products(name,choice,limit=1):
  results=process.extract(name,choice,limit=limit)
  return results[0][0] if results else None

In [23]:
dx['Matched_Product'] = dx['Product_Name'].apply(lambda x: match_products(x,df['Product_Name'].tolist()))

In [24]:
merged_df = pd.merge(df, dx, left_on='Product_Name',right_on='Matched_Product' ,how='inner', suffixes=('_ds1', '_ds2'))

In [25]:
merged_df

,Product_id_ds1,Product_Name_ds1,Rating_ds1,Review_Count_ds1,Product_id_ds2,Product_Name_ds2,Rating_ds2,Review_Count_ds2,Matched_Product
0,p1254,samsung mobile-g56,8.3,1214,mp1254,samsung mobile-j56,7.3,1114,samsung mobile-g56
1,p1254,samsung mobile-g56,8.3,1214,mp1682,samsung mobile-v56,5.2,721,samsung mobile-g56
2,p1852,vivo-1213,4.2,501,mp1852,vivo-1213,3.2,401,vivo-1213
3,p1987,samsung mobile-h56,5.4,623,mp1987,samsung mobile-h56,4.4,523,samsung mobile-h56
4,p1365,vivo-v1-pro,8.3,1211,mp1365,vivo-v1-pro,7.3,1111,vivo-v1-pro
5,p2575,nokia1100,8.4,1235,mp2575,nokia1100,7.4,1135,nokia1100
6,p1152,samsung mobile-a5,7.5,1101,mp1152,samsung mobile-b5,6.5,1001,samsung mobile-a5
7,p6314,vivo-z1-pro,8.9,2010,mp6314,vivo-z1-pro,7.9,1910,vivo-z1-pro
8,p4251,lg-pro-max,6.2,965,mp4251,lg-max-pro,5.2,865,lg-pro-max
9,p9851,one plus 2,8.6,2525,mp9851,one plus 1,7.6,2425,one plus 2


In [26]:
merged_df.columns

Index(['Product_id_ds1', 'Product_Name_ds1', 'Rating_ds1', 'Review_Count_ds1',
       'Product_id_ds2', 'Product_Name_ds2', 'Rating_ds2', 'Review_Count_ds2',
       'Matched_Product'],
      dtype='object')

In [27]:
x=merged_df['Review_Count_ds1']+merged_df['Review_Count_ds2']

In [28]:
merged_df['Combined_Rating'] = (merged_df['Rating_ds1'] * merged_df['Review_Count_ds1']+merged_df['Rating_ds2'] * merged_df['Review_Count_ds2'] ) / x


In [29]:
merged_df['Recommendation_Score']= 0.7* merged_df['Combined_Rating'] + 0.3*merged_df['Rating_ds1']

In [30]:
merged_df.head()

,Product_id_ds1,Product_Name_ds1,Rating_ds1,Review_Count_ds1,Product_id_ds2,Product_Name_ds2,Rating_ds2,Review_Count_ds2,Matched_Product,Combined_Rating,Recommendation_Score
0,p1254,samsung mobile-g56,8.3,1214,mp1254,samsung mobile-j56,7.3,1114,samsung mobile-g56,7.821478,7.965034
1,p1254,samsung mobile-g56,8.3,1214,mp1682,samsung mobile-v56,5.2,721,samsung mobile-g56,7.144910,7.491437
2,p1852,vivo-1213,4.2,501,mp1852,vivo-1213,3.2,401,vivo-1213,3.755432,3.888803
3,p1987,samsung mobile-h56,5.4,623,mp1987,samsung mobile-h56,4.4,523,samsung mobile-h56,4.943630,5.080541
4,p1365,vivo-v1-pro,8.3,1211,mp1365,vivo-v1-pro,7.3,1111,vivo-v1-pro,7.821533,7.965073


In [31]:
best_products = merged_df.sort_values(by='Recommendation_Score', ascending=False)

In [32]:
print(best_products[['Matched_Product','Combined_Rating' ,'Recommendation_Score']].head(10))

       Matched_Product  Combined_Rating  Recommendation_Score
13      one plus x pro         8.707901              8.855531
7          vivo-z1-pro         8.412755              8.558929
10          one plus 2         8.241458              8.349020
9           one plus 2         8.110101              8.257071
5            nokia1100         7.921097              8.064768
4          vivo-v1-pro         7.821533              7.965073
0   samsung mobile-g56         7.821478              7.965034
1   samsung mobile-g56         7.144910              7.491437
14           nokia1221         7.026042              7.168229
6    samsung mobile-a5         7.023787              7.166651


In [33]:
best_products[['Matched_Product', 'Combined_Rating', 'Recommendation_Score']].to_csv('best_products.csv', index=False)

In [34]:
rp=pd.read_csv('best_products.csv')

In [35]:
rp.head()

,Matched_Product,Combined_Rating,Recommendation_Score
0,one plus x pro,8.707901,8.855531
1,vivo-z1-pro,8.412755,8.558929
2,one plus 2,8.241458,8.349020
3,one plus 2,8.110101,8.257071
4,nokia1100,7.921097,8.064768
